In [5]:
from os import path
import pandas as pd
import numpy as np
import pickle

pd.set_option('display.max_columns', None)

In [2]:
#!jupyter nbconvert --to script 1-preprocessing.ipynb

In [12]:
import warnings

# Disable typecast warnings
warnings.filterwarnings('ignore')

na_values=['NA', 'nan']

parse_dates = ['fecha_alta', 'ult_fec_cli_1t']
dtype = {'tiprel_1mes': np.str, 'indrel_1mes':np.str}

iter_csv = pd.read_csv('../data/raw/train.csv.zip', na_values = na_values, skipinitialspace = True, 
                       iterator=True, chunksize=100000, parse_dates=parse_dates, dtype=dtype)

corrupted = pd.concat([chunk for chunk in iter_csv])
corrupted.sort_values(['ncodpers', 'fecha_dato'], inplace=True)
corrupted.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,2015-02-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0
2,2015-03-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0
3,2015-04-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0
4,2015-05-28,15889,F,ES,V,56.0,1995-01-16,0.0,245.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,MADRID,1.0,326124.9,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0


In [13]:
corrupted.drop_duplicates(['ncodpers'], keep='last', inplace=True)

In [4]:
products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [6]:
personal_products = {}

def unique(arr):
    seen = set()
    seen_add = seen.add
    return [x for x in arr if not (x in seen or seen_add(x))]

# this function checks whether any product was bought in provided months a year ago.
def month_last_year(user_history, months):  
    active = []
    
    for m in months:
        curr = '2015-0%d-28' % m
        entry = user_history[user_history.fecha_dato == curr]
        if len(entry) > 0:

            for p in products:
                if entry.iloc[0][p] == 1:
                    active.append(p)
                
    return active
   
# this function collects products which where used more than {percent} of time.
# here {percent} is a fraction from 0 to 1 inclusive. 50% == 0.5
def popular_products(user_history, percent):
    active = []

    for p in products:
        curr = user_history[p][user_history[p].notnull()].values
        usage = (curr == 1).sum()/curr.size
        if usage >= percent:
            active.append(p)
    
    return active

i = 0

def grab_personal(rows):
    user_history = rows.copy(deep=True)
    user_history.reset_index(drop=True, inplace=True)
    
    global i
    
    if i % 10000 == 0:
        print('iloc', user_history.iloc[0]['ncodpers'])
        print('processing entry #%d' % i)
    i += 1

    potential = []
    potential.extend(month_last_year(user_history, [6,5,7]))
    potential.extend(popular_products(user_history, 0.5))

    active_prods = user_history.iloc[-1][products]

    for p in products:
        if active_prods[p] == 1:
            potential = [e for e in potential if e != p]
            
    personal_products[user_history.iloc[0]['ncodpers']] = unique(potential)

In [6]:
PERSONAL_PRODS = '../data/processed/personal.pickle'

if path.isfile(PERSONAL_PRODS):
    print("Loading existing pickle")
    personal_products = pickle.load(open(PERSONAL_PRODS, "rb" ))
else:
    print("Calculating personal data. Takes about 9hrs")
    corrupted.groupby(['ncodpers']).apply(grab_personal)
    
    with open('../data/processed/personal.pickle', 'wb') as f:
        pickle.dump(personal_products, f)

Loading existing pickle


In [14]:
# Remove corrupted entries
print('%d users before cleaning' % len(corrupted.ncodpers.unique()))
corrupted = corrupted[corrupted.age.notnull()]
corrupted = corrupted[corrupted.sexo.notnull()]
print('%d users after cleaning' % len(corrupted.ncodpers.unique()))
corrupted.drop('nomprov', axis=1, inplace=True) # Drop province name. It is 100% useless
corrupted.drop_duplicates(['ncodpers'], keep='last', inplace=True)

956645 users before cleaning
949609 users after cleaning


In [15]:
corrupted.sort_values(['ncodpers', 'fecha_dato'], inplace=True)
corrupted.drop_duplicates(['ncodpers'], keep='last', inplace=True)
corrupted.reset_index(drop=True, inplace=True)
X_train = corrupted

In [16]:
X_train.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2016-05-28,15889,F,ES,V,56.0,1995-01-16,0.0,255.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,1.0,326124.90,01 - TOP,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,2016-05-28,15890,A,ES,V,63.0,1995-01-16,0.0,256.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,1.0,71461.20,01 - TOP,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,2015-08-28,15891,N,ES,H,59.0,2015-07-28,0.0,246.0,99.0,2015-08-05,1,A,S,N,N,KAT,N,1.0,28.0,0.0,NaN,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2016-05-28,15892,F,ES,H,62.0,1995-01-16,0.0,256.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,1.0,430477.41,01 - TOP,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,2016-05-28,15893,N,ES,V,63.0,1997-10-03,0.0,256.0,1.0,NaT,1,A,S,N,N,KAT,N,1.0,28.0,1.0,430477.41,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [17]:
def sub_code_and_toint(text):
    return 0.0 if pd.isnull(text) else float(str(text)[1:2])
    
def preprocess_features(X):
    # "converting dates to timestamps"
    X.fecha_alta = X.fecha_alta.astype('int64')//1e9
    X.fecha_alta[X.fecha_alta < 0] = 0

    X.ult_fec_cli_1t = X.ult_fec_cli_1t.astype('int64')//1e9
    X.ult_fec_cli_1t[X.ult_fec_cli_1t < 0] = 0

    # "ind_empleado"
    ind_empleado = pd.get_dummies(X.ind_empleado, prefix = 'ind_empleado')
    X = pd.concat([X, ind_empleado], axis=1)

    # "tiprel_1mes"
    tiprel_1mes = pd.get_dummies(X.tiprel_1mes, prefix = 'tiprel_1mes')
    X = pd.concat([X, tiprel_1mes], axis=1)        
    
    X.segmento = X.segmento.apply(sub_code_and_toint)
    
    X.sexo.replace(to_replace=['V', 'H'], value=[1.0, 0.0], inplace=True)
    X.renta.replace(to_replace=[float('nan')], value=[0.], inplace=True)
    X.indrel.replace(to_replace=[99.], value=[0.], inplace=True)
    X.indext.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)
    X.indresi.replace(to_replace=['S', 'N'], value=[1.0, 0.0], inplace=True)    
    X.indfall.replace(to_replace=['S','N'], value=[1.,0.], inplace=True)
    X.cod_prov.replace(to_replace=[float('nan')], value=[0.], inplace=True)
    X.indrel_1mes.replace(to_replace=['P', float('nan')], value=[5., 6.], inplace=True)
    X.canal_entrada.replace(to_replace=[None], value=['nan'], inplace=True)
    
    X.indrel_1mes = pd.to_numeric(X.indrel_1mes)
    
    columns_to_drop = ['conyuemp', 'tipodom', 'fecha_dato', 'ind_empleado', 'tiprel_1mes']
    X.drop(columns_to_drop, axis=1, inplace=True) 
    
    return X

In [18]:
X_train = preprocess_features(X_train)
X_train.head()

,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,indresi,indext,canal_entrada,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R
0,15889,ES,1.0,56.0,790214400,0.0,255.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0,0,0,1,0,0,1,0,0,0
1,15890,ES,1.0,63.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1,1,0,0,0,0,1,0,0,0
2,15891,ES,0.0,59.0,1438041600,0.0,246.0,0.0,1438732800,1.0,1.0,0.0,KAT,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,1,0,1,0,0,0
3,15892,ES,0.0,62.0,790214400,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1,0,0,1,0,0,1,0,0,0
4,15893,ES,1.0,63.0,875836800,0.0,256.0,1.0,0,1.0,1.0,0.0,KAT,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0,0,0,0,1,0,1,0,0,0


### coordinates feature
Load external dictionary of coordinates and replace code with coordinates in dataset

In [19]:
countries = pd.read_csv('../data/external/countries.csv', header=None)
countries.columns=['pais_residencia', 'lat', 'lon']

X_train = X_train.merge(countries, on='pais_residencia')
X_train.sort_values(['ncodpers'], inplace=True)
X_train.reset_index(drop=True, inplace=True)

new_cols_order = ['ncodpers', 'pais_residencia', 'sexo', 'age', 'lat', 'lon', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'ind_empleado_A', 'ind_empleado_B', 'ind_empleado_F', 'ind_empleado_N', 'ind_empleado_S', 'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes_A', 'tiprel_1mes_I', 'tiprel_1mes_P', 'tiprel_1mes_R', 'indresi', 'indext', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
X_train = X_train[new_cols_order]

In [20]:
X_train.head()

,ncodpers,pais_residencia,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,ES,1.0,56.0,40.0,-4.0,790214400,0.0,255.0,1.0,0,0,1,0,0,0,1.0,1,0,0,0,1.0,0.0,0.0,28.0,1.0,326124.90,1.0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0.0,0.0,0
1,15890,ES,1.0,63.0,40.0,-4.0,790214400,0.0,256.0,1.0,1,0,0,0,0,0,1.0,1,0,0,0,1.0,0.0,0.0,28.0,1.0,71461.20,1.0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1.0,1.0,1
2,15891,ES,0.0,59.0,40.0,-4.0,1438041600,0.0,246.0,0.0,0,0,0,1,0,1438732800,1.0,1,0,0,0,1.0,0.0,0.0,28.0,0.0,0.00,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,15892,ES,0.0,62.0,40.0,-4.0,790214400,0.0,256.0,1.0,0,0,1,0,0,0,1.0,1,0,0,0,1.0,0.0,0.0,28.0,1.0,430477.41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0.0,0.0,1
4,15893,ES,1.0,63.0,40.0,-4.0,875836800,0.0,256.0,1.0,0,0,0,1,0,0,1.0,1,0,0,0,1.0,0.0,0.0,28.0,1.0,430477.41,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0.0,0.0,0


In [21]:
X_train.to_csv('../data/processed/train-processed.csv.gz', index=False, compression='gzip')